In [56]:
#!touch 0-l2_reg_cost.py
!chmod +x *.py
!pwd

/home/khouloud/hb/holbertonschool-machine_learning/supervised_learning/0x05-regularization


In [55]:
#!/usr/bin/env python3

import numpy as np
#l2_reg_cost = __import__('0-l2_reg_cost').l2_reg_cost
def l2_reg_cost(cost, lambtha, weights, L, m):
    """ doc """
    sw = 0
    for i in range(1, L):
        sw += (weights['W'+str(i)]).sum()
    l2 = lambtha*sw/(2*m)
    return (cost+l2)

if __name__ == '__main__':
    np.random.seed(0)

    weights = {}
    weights['W1'] = np.random.randn(256, 784)
    weights['W2'] = np.random.randn(128, 256)
    weights['W3'] = np.random.randn(10, 128)

    cost = np.abs(np.random.randn(1))

    print(cost)
    cost = l2_reg_cost(cost, 0.1, weights, 3, 1000)
    print(cost)

[0.41842822]
[8.68513236]
